In [13]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
# import talib as ta
import math
# from numba import jit

from importlib import reload 
from qflib import basic
reload(basic)
global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

In [14]:
""" 数据抽查 sz_max_50 """
def datacheck():
    sum = len(df)
    bins = [-float('inf'), 0, 3, 8, 15, float('inf')] 
    labels = ['0-0', '0-3%', '3-8%',  '8-15%', '>15%']

    df['sz_max_5_bin'] = pd.cut(df['sz_max_5'], bins=bins, labels=labels)
    print( df['sz_max_5_bin'].value_counts()/sum, df['sz_max_5'].mean() )

    df['sz_max_10_bin'] = pd.cut(df['sz_max_10'], bins=bins, labels=labels)
    print( df['sz_max_10_bin'].value_counts()/sum, df['sz_max_10'].mean() )

    df['sz_max_20_bin'] = pd.cut(df['sz_max_20'], bins=bins, labels=labels)
    print( df['sz_max_20_bin'].value_counts()/sum, df['sz_max_20'].mean() )


In [15]:

# 指标库读出基本数据 
# xl_01dgree = 0.00174533 # 0.1度 
# xl_02dgree = 0.00349067 # 0.2度 
# xl_03dgree = 0.00523604 # 0.3度
# xl_dgree = xl_03dgree
# print( 'd60 斜率：', str(xl_dgree) )

# trade_date like '%2023-06%' and \
# sql1 = "SELECT * \
# from day_zhibiao \
# where \
# jx_dg_250>-0.1 and jx_dg_60>-0.1 and \
# jx_days_ud60>=20 and jx_days_ud60<=40 and \
# jx_dg_20>=0 and jx_zs_20>=1 and pct_fl_3_10>1.5 and \
# (pct_fl_1_3>1.5 or k_red3>=3) "

sql2 = "SELECT * \
from day_zhibiao \
where \
jx_dg_250>0 and jx_dg_60>0 and \
jx_dg_20>=0 and jx_zs_60>=1 and pct_fl_5_20>1.2 and \
(pct_fl_1_3>1.2 or k_red) "

print( sql2 )
df = pd.read_sql_query(sql2, conn)

print('shape: ', df.shape, '\n')
datacheck()

SELECT * from day_zhibiao where jx_dg_250>0 and jx_dg_60>0 and jx_dg_20>=0 and jx_zs_60>=1 and pct_fl_5_20>1.2 and (pct_fl_1_3>1.2 or k_red) 
shape:  (149165, 85) 

0-0      0.800556
0-3%     0.103536
3-8%     0.065283
8-15%    0.023685
>15%     0.006939
Name: sz_max_5_bin, dtype: float64 0.8675654426574597
0-0      0.779084
0-3%     0.113552
3-8%     0.070700
8-15%    0.026816
>15%     0.009848
Name: sz_max_10_bin, dtype: float64 1.0129318841899912
0-0      0.757926
0-3%     0.128281
3-8%     0.075722
8-15%    0.027768
>15%     0.010304
Name: sz_max_20_bin, dtype: float64 1.0782669689719437


In [16]:
# 检测数据
col_check = ['symbol', 'trade_date', 'pct_change', 'sz_max_10', \
    'diff', 'dea', \
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_20_60', \
    'sz_max_5','sz_max_20',
     ]
sorting = df.sort_values(['trade_date', 'symbol'], ascending=False)
sorting[df['trade_date']>'2023-07'][col_check]
# sorting.head(5)[col_check]

d:\temp\ipykernel_15696\673781611.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sorting[df['trade_date']>'2023-07'][col_check]


,symbol,trade_date,pct_change,sz_max_10,diff,dea,jx_days_ud60,jx_dg_120,jx_dg_60,jx_dg_20,...,jx_dg_5,jx_zs_5,jx_zs_10,jx_zs_20,jx_zs_60,pct_fl_1_3,pct_fl_3_10,pct_fl_20_60,sz_max_5,sz_max_20
148124,605001,2023-08-11,0.51,0.000000,0.196499,0.127604,0,-0.002701,0.133666,0.357647,...,0.986452,4,4,4,4,0.764957,2.280440,3.520430,0.000000,0.000000
142581,603716,2023-08-11,9.99,0.000000,0.845497,0.428223,0,0.162459,0.484599,1.449800,...,4.096370,4,4,15,12,1.028210,1.301020,2.833850,0.000000,0.000000
139397,603580,2023-08-11,1.87,0.000000,0.046768,0.051849,0,-0.115847,0.074722,0.089978,...,0.327635,1,1,1,20,1.448900,1.527520,1.011150,0.000000,1.766780
137285,603383,2023-08-11,1.29,5.733460,4.382130,3.183290,0,0.202103,0.353198,0.862265,...,0.402982,0,7,17,18,1.493930,0.803444,2.917220,5.733460,5.733460
135796,603318,2023-08-11,-1.61,1.637430,-0.003238,-0.017908,0,-0.023162,0.073356,0.010322,...,0.471445,2,2,2,2,2.267070,5.123410,0.872962,1.637430,1.637430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4826,000531,2023-07-03,2.07,1.897020,0.083123,0.055590,0,0.094885,0.153760,0.076421,...,0.077953,1,6,6,52,1.238300,1.293000,1.135420,1.897020,1.897020
4491,000521,2023-07-03,0.52,0.000000,0.143885,0.114848,0,0.294129,0.326338,0.195855,...,1.015450,5,5,5,113,0.814781,1.619690,0.647445,0.000000,0.000000
3387,000421,2023-07-03,1.85,0.909091,0.146799,0.094743,0,0.073640,0.115484,0.366950,...,-0.062847,1,11,10,10,1.269830,0.738500,4.320920,0.909091,0.909091
2732,000338,2023-07-03,3.45,0.000000,0.160151,0.079634,0,0.110844,0.022162,0.265223,...,0.898612,5,5,14,5,2.785000,1.549760,0.784417,0.000000,0.000000


In [7]:
""" 数据抽查 sz_max_5 """
sum = len(df)
bins = [-float('inf'), 0, 5, 8, float('inf')] 
labels = ['0-0', '0-5%', '5-8%', '>8%']
df['sz_max_5_bin'] = pd.cut(df['sz_max_5'], bins=bins, labels=labels)
# print( df['sz_max_10_bin'].value_counts() )
df['sz_max_5_bin'].value_counts()/sum, df['sz_max_5'].mean()

(0-0     0.847351
 0-5%    0.107820
 >8%     0.022451
 5-8%    0.022378
 Name: sz_max_5_bin, dtype: float64,
 0.6410563218123744)

In [8]:
""" 数据抽查 sz_max_10 """
# sum = len(df)
# bins = [-float('inf'), 0,3,8, float('inf')] 
# labels = ['0', '0-3', '3-8', '>8']
# df['sz_max_10_bin'] = pd.cut(df['sz_max_10'], bins=bins, labels=labels)
# df['sz_max_10_bin'].value_counts(), df['sz_max_10_bin'].value_counts()/sum

' 数据抽查 sz_max_10 '

In [9]:
""" 数据抽查 sz_max_20 """
sum = len(df)
bins = [-float('inf'), 0, 5, 12, float('inf')] 
labels = ['<=0', '0-5','5-12', '>12']
df['sz_max_20_bin'] = pd.cut(df['sz_max_20'], bins=bins, labels=labels)
df['sz_max_20_bin'].value_counts(), df['sz_max_20_bin'].value_counts()/sum, df['sz_max_20'].mean()

(<=0     112447
 0-5      27807
 5-12      9405
 >12       2186
 Name: sz_max_20_bin, dtype: int64,
 <=0     0.740538
 0-5     0.183128
 5-12    0.061938
 >12     0.014396
 Name: sz_max_20_bin, dtype: float64,
 1.1119300164872072)

In [10]:
# 涨幅阶梯化

def jieti_5(df):
    # rate = 0, 1, 2, 3
    if df['sz_max_5']==0: rate = 0        
    if df['sz_max_5']>0 and df['sz_max_5']<=3: rate = 1          
    if df['sz_max_5']>3 and df['sz_max_5']<=8: rate = 2          
    if df['sz_max_5']>8: rate = 3        
    return rate
df['sz_jt_5'] = df.apply(jieti_5, axis=1) 

def jieti_10(df):
    # rate = 0, 1, 2, 3
    if df['sz_max_10']==0: rate = 0        
    if df['sz_max_10']>0 and df['sz_max_10']<=3: rate = 1          
    if df['sz_max_10']>3 and df['sz_max_10']<=8: rate = 2          
    if df['sz_max_10']>8: rate = 3
    return rate
df['sz_jt_10'] = df.apply(jieti_10, axis=1) 

def jieti_20(df):
    # rate = 0, 1, 2
    if df['sz_max_20'] <= 8: rate = 0        
    if df['sz_max_20']>8 and df['sz_max_10']<=15: rate = 1        
    if df['sz_max_20']>15: rate = 2        
    return rate
df['sz_jt_20'] = df.apply(jieti_20, axis=1) 

In [11]:
""" 临时输出sample """ 
col_sample = ['symbol', 'trade_date', 'pct_change', 'sz_max_10', \
    'diff', 'dea', \
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_20_60', \
    'sz_jt_20',
     ]
df[col_sample].head(100).to_csv('ds_qm.csv', index=False) 

In [12]:
# 数据监测
# df[ ['sz_max_5', 'sz_jt_5', 'sz_max_10','sz_jt_10', 'sz_max_20', 'sz_jt_20']].head(20)
# df[ ['sz_max_20', 'sz_jt_20']].head(20)

In [ ]:
""" 形成数据集 """

# 0-1类训练目标s：
df['test00'] = df['sz_max_20'] > 10
df['test5_8'] = df['sz_max_5'] > 8
df['test10_10'] = df['sz_max_10'] > 10
df['test20_10'] = df['sz_max_20'] > 10
df['test20_20'] = df['sz_max_20'] > 20
df['test20_30'] = df['sz_max_20'] > 30
# 多分类训练目标
df['class0'] = df['sz_jt_10']
# 确定输出列  
col_list_ds = ['ts_code', 'symbol', 'trade_date', 'pct_change',  \
    'diff', 'dea', 'bar',\
    'jx_days_ud60',\
    'jx_dg_120', 'jx_dg_60', 'jx_dg_20', 'jx_dg_10', 'jx_dg_5',\
    'jx_zs_5', 'jx_zs_10', 'jx_zs_20', 'jx_zs_60', \
    'pct_fl_1_3', 'pct_fl_3_10', 'pct_fl_5_20', 'pct_fl_20_60', \
    'sz_max_5','sz_max_10','sz_max_20',
    'test00', 'test5_8', 'test10_10', 'test20_10',  'test20_20', 'test20_30', \
    'class0','sz_jt_5','sz_jt_10','sz_jt_20' ]
df_ds = df[col_list_ds].copy()
# 'test00', 'test5_8', 'test10_10', 'test20_10',  'test20_20', 'test20_30', \
# df_ds[ df_ds['test00'] ].shape, df_ds[ df_ds['test5_8'] ].shape, df_ds[ df_ds['test10_10'] ].shape, df_ds[df_ds['test20_10']].shape, df_ds[df_ds['test20_20']].shape, df_ds[ df_ds['test20_30'] ].shape

In [ ]:
""" 输出到mysql """
# 删除旧数据
sql = "truncate table ds_qf60"
basic.exec_sql(conn, sql)
# 输出新数据
df_ds.to_sql('ds_qf60', con=conn, index=False, if_exists='append', chunksize=2000) 

In [ ]:
# name: 数据库中的表名
# con: 与read_sql中相同，数据库连接的驱动
# if_exits： 当数据库中的这个表存在的时候，采取的措施是什么，包括三个值，默认为fail
# fail，若表存在，则不进行数据表写入的相关操作
# replace：若表存在，将数据库表中的数据覆盖；
# append：若表存在，将数据写到原表的后面。
# index：是否将df的index单独写到一列中
# index_label:指定列作为df的index输出，此时index为True
# dtype: 指定列的输出到数据库中的数据类型。字典形式储存：{column_name: sql_dtype}